#  Greedy Algorithm


In [3]:
from datetime import datetime, timedelta
import pandas as pd
import random
from collections import defaultdict
import heapq
import plotly.express as px

# FINAL FINAL FINAL FINAL FINAL AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
# Define divisions - set menu items 
divisions = {
    "Baking": ["Bread", "Croissant", "Danish Pastry", "Baguette"],
    "Pastry": ["Cake", "Pie", "Cookies", "Brownies"],
    "Confectionery": ["Macarons", "Cupcakes", "Muffins", "Donuts", "Bagels"],
    "Butchery": ["Quiche"]
}

# Define total available machinery
total_machinery = {
    "Mixer": 4,
    "Proofer": 2,
    "Oven": 3
}

# Define unit times for each machine and item (in minutes)
unit_times = {
    "Mixer": {
        "Bread": 10,
        "Croissant": 8,
        "Danish Pastry": 12,
        "Cake": 15,
        "Pie": 20,
        "Cookies": 5,
        "Brownies": 8,
        "Macarons": 3,
        "Cupcakes": 4,
        "Muffins": 5,
        "Donuts": 6,
        "Bagels": 7,
        "Quiche": 10,
        "Baguette": 15
    },
    "Oven": {
        "Bread": 45,
        "Croissant": 25,
        "Danish Pastry": 30,
        "Cake": 60,
        "Pie": 40,
        "Cookies": 15,
        "Brownies": 20,
        "Macarons": 10,
        "Cupcakes": 12,
        "Muffins": 10,
        "Donuts": 8,
        "Bagels": 15,
        "Quiche": 30,
        "Baguette": 15
    },
    "Proofer": {
        "Bread": 60,
        "Croissant": 30,
        "Danish Pastry": 45,
        "Cake": 0,
        "Pie": 0,
        "Cookies": 0,
        "Brownies": 0,
        "Macarons": 0,
        "Cupcakes": 0,
        "Muffins": 0,
        "Donuts": 15,
        "Bagels": 45,
        "Quiche": 0,
        "Baguette": 15
    }
}

# Define machine downtime (in minutes)
downtime = {
    "Mixer": 15,
    "Oven": 30,
    "Proofer": 10,
}

def get_special_menu_input():
    special_items = []
    while True:
        item_name = input("Enter special menu item (type 'done' to finish): ")
        if item_name.lower() == "done":
            break

        print("Enter Division Responsible for this Menu item:")
        for i, division_name in enumerate(divisions.keys(), 1):  # Print divisions with numbers
            print(f"{i}. {division_name}")

        try:
            division_choice = int(input("Choose a division (1-4): "))
            if division_choice not in range(1, len(divisions) + 1):  # Check against valid range
                print("Invalid division. Please enter a valid division.")
                continue
            division = list(divisions.keys())[division_choice - 1]  # Get division name by index
        except (ValueError, IndexError):
            print("Invalid input. Please enter a number between 1 and 4.")
            continue

        processing_times = {}
        for machine in total_machinery.keys():
            try:
                processing_time = int(input(f"Enter processing time for {machine} in minutes: "))
                if processing_time < 0:
                    raise ValueError
                processing_times[machine] = processing_time
            except ValueError:
                print("Invalid input. Please enter a valid positive number.")
                continue

        special_items.append({
            "name": item_name,
            "division": division,
            "processing_times": processing_times
        })

    return special_items


def schedule_tasks(date, special_items_input):
    """Schedules tasks for a given date, considering shared machinery, enforcing machine order for all divisions, including machine downtime, and minimizing unnecessary waiting."""
    schedule = defaultdict(lambda: defaultdict(list))
    start_time = datetime.combine(date.date(), datetime.now().replace(hour=4, minute=0).time())

    # Initialize machine_times with tuples of (time, machine_index)
    machine_times = {machine: [(start_time, i) for i in range(total_machinery[machine])] for machine in total_machinery}

    # Convert to heaps
    for machine in machine_times:
        heapq.heapify(machine_times[machine])

    machine_order = ["Mixer", "Proofer", "Oven"]  # Define the machine order

    # Create a dictionary to store tasks for each item
    item_tasks = defaultdict(list)
    for division_name, division_items in divisions.items():
        all_items = division_items + [f"Special: {item['name']}" for item in special_items_input if item['division'] == division_name]
        for item in all_items:
            item_name = item if not item.startswith("Special: ") else item[9:]
            processing_time = {machine: unit_times[machine].get(item_name, 0) for machine in machine_order}
            if "Special: " in item:
                special_item = next((i for i in special_items_input if i["name"] == item_name), None)
                if special_item:
                    processing_time.update(special_item["processing_times"])
            for i, machine in enumerate(machine_order):
                if processing_time[machine] > 0:
                    item_tasks[item].append((division_name, machine, processing_time[machine]))  # Store tasks for each item

    task_queue = []  # Initialize an empty task queue

    # Initially add the first task for each item to the queue
    for item, tasks in item_tasks.items():
        task_queue.append((start_time, item, tasks[0][0], tasks[0][1], 0))  # Add first task for each item

    heapq.heapify(task_queue)  # Sort task queue initially ------- INITIAL TASK QUEUE INDEPENDENT

    # Initialize a dictionary to track machine usage
    machine_index_used = {machine: [False] * total_machinery[machine] for machine in total_machinery}

    while task_queue:
        task_time, item, division_name, machine, task_index = heapq.heappop(task_queue)

        # Find the earliest available machine of this type
        machine_available_time, machine_index = heapq.heappop(machine_times[machine])

        # Apply downtime if the machine has been used BEFORE this task
        if machine_index_used[machine][machine_index]:
            task_start_time = max(task_time, machine_available_time + timedelta(minutes=downtime[machine]))
        else:
            task_start_time = max(task_time, machine_available_time)
        #Mark Current Machine as Used
        machine_index_used[machine][machine_index] = True



        # Schedule the task
        schedule[division_name][machine].append((task_start_time, item, machine_index + 1))

        # Update machine availability, incorporating processing time and downtime
        task_completion_time = task_start_time + timedelta(minutes=item_tasks[item][task_index][2]) # task completion time WITHOUT downtime

        #Calculate: downtime START
        downtime_start_time = task_completion_time

        # Add downtime to machine availability for next task on this machine.
        
        heapq.heappush(machine_times[machine], # biang kerok 
                       (downtime_start_time, machine_index))  # Update machine_times with downtime

        # Add the next task for this item to the queue, if any
        next_task_index = task_index + 1
        if next_task_index < len(item_tasks[item]):
            next_task = item_tasks[item][next_task_index]
            # task_time for next task is the completion time of the current task WITHOUT downtime
            next_task_time = task_completion_time  # Use completion time without downtime here
            task_queue.append((next_task_time, item, next_task[0], next_task[1], next_task_index))

        heapq.heapify(task_queue)  # Sort task queue after adding each task

    return schedule

def print_schedule(schedule, date):
    """Prints the schedule for a given date."""
    print(f"\nSchedule for {date.strftime('%Y-%m-%d')}:")
    for division, machines in schedule.items():
        print(f"  {division}:")
        for machine, tasks in machines.items():
            print(f"    {machine}:")
            for task_time, item, machine_number in tasks:
                print(f"      - {task_time.strftime('%H:%M')}: {item} (Machine {machine_number})")


def create_gantt_chart(schedule, date):
    """
    Creates a Gantt chart from the schedule_tasks output, including downtime in red.

    Args:
        schedule (dict): The output from schedule_tasks.
        date (datetime): The date for which the schedule is generated.
    """
    df = []

    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Calculate the finish time based on unit_times
                finish_time = task_time + timedelta(minutes=unit_times.get(machine, {}).get(item, 0))

                # Append tasks to the DataFrame
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=task_time,
                    Finish=finish_time,
                    Item=item,
                    Type="Task"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

                # Add downtime after the task
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=finish_time,
                    Finish=finish_time + timedelta(minutes=downtime[machine]),
                    Item="Downtime",
                    Type="Downtime"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

    # Convert to DataFrame
    df = pd.DataFrame(df)

    # Ensure Start and Finish are datetime objects
    df['Start'] = pd.to_datetime(df['Start'])
    df['Finish'] = pd.to_datetime(df['Finish'])

    # Create the Gantt chart using Plotly Express
    # Define the colors for tasks and downtime
    color_map = {"Downtime": "grey"}  # Set downtime color to black

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Item",
                      color_discrete_map=color_map,
                      title=f"Machine Workload Visualization (Greedy Algorithm) for {date.strftime('%Y-%m-%d')}",
                      labels={"Item": "Menu Item", "Machine": "Machine Index"})
    # Update layout for better visibility
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_traces(marker=dict(line=dict(width=0)))

    fig.show()

# Example usage
today = datetime.now()
next_week = today + timedelta(days=7)

special_items_input = get_special_menu_input()

for date in [today, next_week]:
    schedule = schedule_tasks(date, special_items_input)
    create_gantt_chart(schedule, date)

## +BIG O, time & space complexity

In [5]:
from datetime import datetime, timedelta
import pandas as pd
import random
from collections import defaultdict
import heapq
import plotly.express as px

import time
import tracemalloc

# FINAL FINAL FINAL FINAL FINAL AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
# Define divisions - set menu items 
divisions = {
    "Baking": ["Bread", "Croissant", "Danish Pastry"],
    "Pastry": ["Cake", "Pie", "Cookies", "Brownies"],
    "Confectionery": ["Macarons", "Cupcakes", "Muffins", "Donuts", "Bagels"],
    "Butchery": ["Quiche"]
}

# Define total available machinery
total_machinery = {
    "Mixer": 4,
    "Proofer": 2,
    "Oven": 3
}

# Define unit times for each machine and item (in minutes)
unit_times = {
    "Mixer": {
        "Bread": 10,
        "Croissant": 8,
        "Danish Pastry": 12,
        "Cake": 15,
        "Pie": 20,
        "Cookies": 5,
        "Brownies": 8,
        "Macarons": 3,
        "Cupcakes": 4,
        "Muffins": 5,
        "Donuts": 6,
        "Bagels": 7,
        "Quiche": 10
    },
    "Oven": {
        "Bread": 45,
        "Croissant": 25,
        "Danish Pastry": 30,
        "Cake": 60,
        "Pie": 40,
        "Cookies": 15,
        "Brownies": 20,
        "Macarons": 10,
        "Cupcakes": 12,
        "Muffins": 10,
        "Donuts": 8,
        "Bagels": 15,
        "Quiche": 30
    },
    "Proofer": {
        "Bread": 60,
        "Croissant": 30,
        "Danish Pastry": 45,
        "Cake": 0,
        "Pie": 0,
        "Cookies": 0,
        "Brownies": 0,
        "Macarons": 0,
        "Cupcakes": 0,
        "Muffins": 0,
        "Donuts": 15,
        "Bagels": 45,
        "Quiche": 0
    }
}

# Define machine downtime (in minutes)
downtime = {
    "Mixer": 15,
    "Oven": 30,
    "Proofer": 10,
}

def get_special_menu_input():
    special_items = []
    while True:
        item_name = input("Enter special menu item (type 'done' to finish): ")
        if item_name.lower() == "done":
            break

        print("Enter Division Responsible for this Menu item:")
        for i, division_name in enumerate(divisions.keys(), 1):  # Print divisions with numbers
            print(f"{i}. {division_name}")

        try:
            division_choice = int(input("Choose a division (1-4): "))
            if division_choice not in range(1, len(divisions) + 1):  # Check against valid range
                print("Invalid division. Please enter a valid division.")
                continue
            division = list(divisions.keys())[division_choice - 1]  # Get division name by index
        except (ValueError, IndexError):
            print("Invalid input. Please enter a number between 1 and 4.")
            continue

        processing_times = {}
        for machine in total_machinery.keys():
            try:
                processing_time = int(input(f"Enter processing time for {machine} in minutes: "))
                if processing_time < 0:
                    raise ValueError
                processing_times[machine] = processing_time
            except ValueError:
                print("Invalid input. Please enter a valid positive number.")
                continue

        special_items.append({
            "name": item_name,
            "division": division,
            "processing_times": processing_times
        })

    return special_items


def schedule_tasks(date, special_items_input):
    """Schedules tasks for a given date, considering shared machinery, enforcing machine order for all divisions, including machine downtime, and minimizing unnecessary waiting."""
    schedule = defaultdict(lambda: defaultdict(list))
    start_time = datetime.combine(date.date(), datetime.now().replace(hour=4, minute=0).time())

    # Initialize machine_times with tuples of (time, machine_index)
    machine_times = {machine: [(start_time, i) for i in range(total_machinery[machine])] for machine in total_machinery}

    # Convert to heaps
    for machine in machine_times:
        heapq.heapify(machine_times[machine])

    machine_order = ["Mixer", "Proofer", "Oven"]  # Define the machine order

    # Create a dictionary to store tasks for each item
    item_tasks = defaultdict(list)
    for division_name, division_items in divisions.items():
        all_items = division_items + [f"Special: {item['name']}" for item in special_items_input if item['division'] == division_name]
        for item in all_items:
            item_name = item if not item.startswith("Special: ") else item[9:]
            processing_time = {machine: unit_times[machine].get(item_name, 0) for machine in machine_order}
            if "Special: " in item:
                special_item = next((i for i in special_items_input if i["name"] == item_name), None)
                if special_item:
                    processing_time.update(special_item["processing_times"])
            for i, machine in enumerate(machine_order):
                if processing_time[machine] > 0:
                    item_tasks[item].append((division_name, machine, processing_time[machine]))  # Store tasks for each item

    task_queue = []  # Initialize an empty task queue

    # Initially add the first task for each item to the queue
    for item, tasks in item_tasks.items():
        task_queue.append((start_time, item, tasks[0][0], tasks[0][1], 0))  # Add first task for each item

    heapq.heapify(task_queue)  # Sort task queue initially ------- INITIAL TASK QUEUE INDEPENDENT

    # Initialize a dictionary to track machine usage
    machine_index_used = {machine: [False] * total_machinery[machine] for machine in total_machinery}

    while task_queue:
        task_time, item, division_name, machine, task_index = heapq.heappop(task_queue)

        # Find the earliest available machine of this type
        machine_available_time, machine_index = heapq.heappop(machine_times[machine])

        # Apply downtime if the machine has been used BEFORE this task
        if machine_index_used[machine][machine_index]:
            task_start_time = max(task_time, machine_available_time + timedelta(minutes=downtime[machine]))
        else:
            task_start_time = max(task_time, machine_available_time)
        #Mark Current Machine as Used
        machine_index_used[machine][machine_index] = True



        # Schedule the task
        schedule[division_name][machine].append((task_start_time, item, machine_index + 1))

        # Update machine availability, incorporating processing time and downtime
        task_completion_time = task_start_time + timedelta(minutes=item_tasks[item][task_index][2]) # task completion time WITHOUT downtime

        #Calculate: downtime START
        downtime_start_time = task_completion_time

        # Add downtime to machine availability for next task on this machine.
       
        heapq.heappush(machine_times[machine], # biang kerok 
                       (downtime_start_time, machine_index))  # Update machine_times with downtime

        # Add the next task for this item to the queue, if any
        next_task_index = task_index + 1
        if next_task_index < len(item_tasks[item]):
            next_task = item_tasks[item][next_task_index]
            # task_time for next task is the completion time of the current task WITHOUT downtime
            next_task_time = task_completion_time  # Use completion time without downtime here
            task_queue.append((next_task_time, item, next_task[0], next_task[1], next_task_index))

        heapq.heapify(task_queue)  # Sort task queue after adding each task

    return schedule

def print_schedule(schedule, date):
    """Prints the schedule for a given date."""
    print(f"\nSchedule for {date.strftime('%Y-%m-%d')}:")
    for division, machines in schedule.items():
        print(f"  {division}:")
        for machine, tasks in machines.items():
            print(f"    {machine}:")
            for task_time, item, machine_number in tasks:
                print(f"      - {task_time.strftime('%H:%M')}: {item} (Machine {machine_number})")


def create_gantt_chart(schedule, date):
    """
    Creates a Gantt chart from the schedule_tasks output, including downtime in red.

    Args:
        schedule (dict): The output from schedule_tasks.
        date (datetime): The date for which the schedule is generated.
    """
    df = []

    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Calculate the finish time based on unit_times
                finish_time = task_time + timedelta(minutes=unit_times.get(machine, {}).get(item, 0))

                # Append tasks to the DataFrame
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=task_time,
                    Finish=finish_time,
                    Item=item,
                    Type="Task"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

                # Add downtime after the task
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=finish_time,
                    Finish=finish_time + timedelta(minutes=downtime[machine]),
                    Item="Downtime",
                    Type="Downtime"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

    # Convert to DataFrame
    df = pd.DataFrame(df)

    # Ensure Start and Finish are datetime objects
    df['Start'] = pd.to_datetime(df['Start'])
    df['Finish'] = pd.to_datetime(df['Finish'])

    # Create the Gantt chart using Plotly Express
    # Define the colors for tasks and downtime
    color_map = {"Downtime": "grey"}  # Set downtime color to black

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Item",
                      color_discrete_map=color_map,
                      title=f"Machine Workload Visualization (Greedy Algorithm) for {date.strftime('%Y-%m-%d')}",
                      labels={"Item": "Menu Item", "Machine": "Machine Index"})
    # Update layout for better visibility
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_traces(marker=dict(line=dict(width=0)))

    fig.show()

def measure_total_performance(schedule_function, dates, *args):
    """
    Measures the total time and space complexity of the scheduling algorithm for multiple dates.

    Args:
        schedule_function (function): The scheduling function to measure.
        dates (list): List of dates to run the scheduling function for.
        *args: Additional arguments to pass to the scheduling function.

    Returns:
        dict: Total time taken (in nanoseconds) and peak memory usage (in bytes).
    """
    total_time_ms = 0
    total_peak_memory = 0

    for date in dates:
        # Start measuring memory
        tracemalloc.start()

        # Start measuring time in nanoseconds
        start_time_ms = time.time()

        # Execute the scheduling function
        schedule_function(date, *args)

        # End measuring time in nanoseconds
        end_time_ms = time.time()

        # Stop measuring memory
        current, peak_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        # Accumulate total time and memory
        total_time_ms += ((end_time_ms - start_time_ms) * 1000)
        total_peak_memory = max(total_peak_memory, peak_memory)  # Use peak memory across all runs

    return {
        "Total Time Taken (ms)": total_time_ms,
        "Peak Memory Usage (bytes)": total_peak_memory
    }

# Example usage
today = datetime.now()
next_week = today + timedelta(days=0)

special_items_input = []
dates = [today, next_week]  # List of dates

# Normal
performance = measure_total_performance(schedule_tasks, dates, special_items_input)

# Print total performance metrics
print(f"Greedy Algorithm")
print(f"Total Performance Metrics for {len(dates)} days:")
print(f"  Total Time Taken: {performance['Total Time Taken (ms)']} ms")
print(f"  Peak Memory Usage: {performance['Peak Memory Usage (bytes)']} bytes")

print('\n')
print('==========================================================================')
print('\n')

schedule = schedule_tasks(today, special_items_input)
create_gantt_chart(schedule, today)

Greedy Algorithm
Total Performance Metrics for 2 days:
  Total Time Taken: 9.625673294067383 ms
  Peak Memory Usage: 5636 bytes






# Simulated Annealing

In [4]:
from datetime import datetime, timedelta
import pandas as pd
import random
from collections import defaultdict
import heapq
import plotly.express as px
import math

# FIRSTTTTTTT
# Define divisions and set menu items (exclusive to each division)
divisions = {
    "Baking": ["Bread", "Croissant", "Danish Pastry", "Baguette"],
    "Pastry": ["Cake", "Pie", "Cookies", "Brownies"],
    "Confectionery": ["Macarons", "Cupcakes", "Muffins", "Donuts", "Bagels"],
    "Butchery": ["Quiche"]
}

# Define total available machinery
total_machinery = {
    "Mixer": 4,
    "Proofer": 2,
    "Oven": 3
}

# Define unit times for each machine and item (in minutes)
unit_times = {
    "Mixer": {
        "Bread": 10,
        "Croissant": 8,
        "Danish Pastry": 12,
        "Cake": 15,
        "Pie": 20,
        "Cookies": 5,
        "Brownies": 8,
        "Macarons": 3,
        "Cupcakes": 4,
        "Muffins": 5,
        "Donuts": 6,
        "Bagels": 7,
        "Quiche": 10,
        "Baguette": 15
    },
    "Oven": {
        "Bread": 45,
        "Croissant": 25,
        "Danish Pastry": 30,
        "Cake": 60,
        "Pie": 40,
        "Cookies": 15,
        "Brownies": 20,
        "Macarons": 10,
        "Cupcakes": 12,
        "Muffins": 10,
        "Donuts": 8,
        "Bagels": 15,
        "Quiche": 30,
        "Baguette": 15
    },
    "Proofer": {
        "Bread": 60,
        "Croissant": 30,
        "Danish Pastry": 45,
        "Cake": 0,
        "Pie": 0,
        "Cookies": 0,
        "Brownies": 0,
        "Macarons": 0,
        "Cupcakes": 0,
        "Muffins": 0,
        "Donuts": 15,
        "Bagels": 45,
        "Quiche": 0,
        "Baguette": 15
    }
}


# Define machine downtime (in minutes)
downtime = {
    "Mixer": 15,
    "Oven": 30,
    "Proofer": 10,
}

def get_special_menu_input():
    special_items = []
    while True:
        item_name = input("Enter special menu item (type 'done' to finish): ")
        if item_name.lower() == "done":
            break

        print("Enter Division Responsible for this Menu item:")
        for i, division_name in enumerate(divisions.keys(), 1):  # Print divisions with numbers
            print(f"{i}. {division_name}")

        try:
            division_choice = int(input("Choose a division (1-4): "))
            if division_choice not in range(1, len(divisions) + 1):  # Check against valid range
                print("Invalid division. Please enter a valid division.")
                continue
            division = list(divisions.keys())[division_choice - 1]  # Get division name by index
        except (ValueError, IndexError):
            print("Invalid input. Please enter a number between 1 and 4.")
            continue

        processing_times = {}
        for machine in total_machinery.keys():
            try:
                processing_time = int(input(f"Enter processing time for {machine} in minutes: "))
                if processing_time < 0:
                    raise ValueError
                processing_times[machine] = processing_time
            except ValueError:
                print("Invalid input. Please enter a valid positive number.")
                continue

        special_items.append({
            "name": item_name,
            "division": division,
            "processing_times": processing_times
        })

    return special_items

def schedule_tasks(divisions, special_items, machines):
    # Initial solution: GACHA SCHEDULE RAWR
    # Pass machine_order to generate_initial_schedule
    machine_order = ["Mixer", "Proofer", "Oven"]  
    current_schedule = generate_initial_schedule(divisions, special_items, machines, machine_order)
    best_schedule = current_schedule
    best_cost = calculate_cost(best_schedule)

    if best_cost is None:
        raise ValueError("Initial best cost is None. Check the calculate_cost function.")

    initial_temp = 1000  # Initial temperature
    cooling_rate = 0.95   # Cooling rate
    temp = initial_temp

    while temp > 1:
        # Generate a new schedule by slightly modify the gacha current schedule
        new_schedule = modify_schedule(current_schedule)

        # Calculate the cost of the new schedule
        new_cost = calculate_cost(new_schedule)

        if new_cost is None:
            raise ValueError("New cost is None. Check the calculate_cost function.")

        # Decide whether to accept the new schedule
        if accept_solution(best_cost, new_cost, temp):
            current_schedule = new_schedule
            if new_cost < best_cost:
                best_schedule = new_schedule
                best_cost = new_cost

        # Cool down the temperature
        temp *= cooling_rate

    return best_schedule

def generate_initial_schedule(divisions, special_items, machines, machine_order):
    schedule = defaultdict(lambda: defaultdict(list))
    start_time = datetime.now().replace(hour=4, minute=0)  # Starting at 4 AM
    machine_order = ["Mixer", "Proofer", "Oven"]  # Define the required machine sequence

    # Machine availability tracking 
    machine_times = {machine: [(start_time, i) for i in range(machines[machine])] for machine in machine_order}

    # Convert to heaps for efficient retrieval of earliest available time
    for machine in machine_times:
        heapq.heapify(machine_times[machine])

    # Combine regular items and special items
    for division, items in divisions.items():
        all_items = items + [item['name'] for item in special_items if item['division'] == division]
        for item in all_items:
            # Schedule the item on the required machines in order
            previous_task_completion_time = start_time  # Initialize for dependency tracking

            for machine in machine_order:
                duration = unit_times[machine].get(item, 0)  # Get processing time
                if duration > 0:  # Schedule only if processing time is greater than 0
                    # Find the earliest available machine of this type
                    machine_available_time, machine_index = heapq.heappop(machine_times[machine])
                    task_start_time = max(previous_task_completion_time, machine_available_time)

                    # Schedule the task
                    schedule[division][machine].append((task_start_time, item, machine_index + 1))

                    # Update machine availability, incorporating the processing time for THIS machine
                    task_completion_time = task_start_time + timedelta(minutes=duration)
                    heapq.heappush(machine_times[machine], (task_completion_time + timedelta(minutes=downtime[machine]), machine_index))

                    # Update previous_task_completion_time for the NEXT task of THIS ITEM
                    previous_task_completion_time = task_completion_time
                    
    # --- Added Check to ensure all items are in the schedule ---
    all_menu_items = [item for sublist in divisions.values() for item in sublist] + [item['name'] for item in special_items]
    scheduled_items = [item for division_schedule in schedule.values() for machine_schedule in division_schedule.values() for _, item, _ in machine_schedule]

    missing_items = set(all_menu_items) - set(scheduled_items)
    if missing_items:
        print("WARNING: The following items were not scheduled:", missing_items)

    return schedule

def modify_schedule(schedule):
    # Convert the schedule to a list of tasks for modification
    tasks = []
    for division, machines in schedule.items():
        for machine, task_list in machines.items():
            tasks.extend(task_list)

    # Randomly swap two tasks
    if len(tasks) > 1:
        idx1, idx2 = random.sample(range(len(tasks)), 2)
        tasks[idx1], tasks[idx2] = tasks[idx2], tasks[idx1]  # Swap tasks

    # Rebuild the schedule from modified tasks
    new_schedule = defaultdict(lambda: defaultdict(list))
    for task in tasks:
        # Extract division and machine information from the original schedule
        division = None
        machine = None
        for div, machines in schedule.items():
            for mach, task_list in machines.items():
                if task in task_list:
                    division = div
                    machine = mach
                    break
            if division is not None:
                break

        if division is not None and machine is not None:
            task_time, item, machine_number = task
            new_schedule[division][machine].append((task_time, item, machine_number))
        else:
            print("WARNING: Task not found in original schedule:", task)

    return new_schedule

def accept_solution(best_cost, new_cost, temp):
    """
    Determines whether to accept a new solution based on the Metropolis criterion.

    Args:
        best_cost (float): The cost of the best solution found so far.
        new_cost (float): The cost of the new solution.
        temp (float): The current temperature.

    Returns:
        bool: True if the new solution is accepted, False otherwise.
    """
    if new_cost < best_cost:
        return True  # Always accept if the new solution is better
    else:
        # Accept with a probability that decreases with temperature
        acceptance_probability = math.exp((best_cost - new_cost) / temp)
        return random.random() < acceptance_probability

def calculate_cost(schedule):
    total_cost = 0  # Initialize total cost

    # Iterate through the schedule to calculate total processing time
    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Get the processing time for the item on the machine
                processing_time = unit_times[machine].get(item, 0)
                # Add processing time to total cost
                total_cost += processing_time

                # Add downtime after each task
                total_cost += downtime[machine]

    return total_cost



def create_gantt_chart(schedule, date):
    """
    Creates a Gantt chart from the schedule_tasks output, including downtime in red.

    Args:
        schedule (dict): The output from schedule_tasks.
        date (datetime): The date for which the schedule is generated.
    """
    df = []

    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Calculate the finish time based on unit_times
                finish_time = task_time + timedelta(minutes=unit_times.get(machine, {}).get(item, 0))

                # Append tasks to the DataFrame
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=task_time,
                    Finish=finish_time,
                    Item=item,
                    Type="Task"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

                # Add downtime after the task
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=finish_time,
                    Finish=finish_time + timedelta(minutes=downtime[machine]),
                    Item="Downtime",
                    Type="Downtime"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

    # Convert to DataFrame
    df = pd.DataFrame(df)  #Convert the list of dictionaries to a Pandas DataFrame

    # Check if the DataFrame is empty
    if df.empty:
        # Create a DataFrame with the needed columns and a single empty row if empty.
        df = pd.DataFrame(columns=["Machine", "Start", "Finish", "Item", "Type"])
        df.loc[0] = [None, None, None, None, None]

    #The rest of your function as it was
    # Ensure Start and Finish are datetime objects
    df['Start'] = pd.to_datetime(df['Start'])
    df['Finish'] = pd.to_datetime(df['Finish'])


    # Create the Gantt chart using Plotly Express
    # Define the colors for tasks and downtime
    color_map = {"Downtime": "grey"}  # Set downtime color to black

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Item",
                      color_discrete_map=color_map,
                      title=f"Machine Workload Visualization (Simulated Annealing) for {date.strftime('%Y-%m-%d')}",
                      labels={"Item": "Menu Item", "Machine": "Machine Index"})
    # Update layout for better visibility
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_traces(marker=dict(line=dict(width=0)))

    fig.show()

def create_dataframe(schedule):
    # Implementation for creating a DataFrame from the schedule
    pass


def print_schedule(schedule, date):
    """Prints the schedule for a given date."""
    print(f"\nSchedule for {date.strftime('%Y-%m-%d')}:")
    for division, machines in schedule.items():
        print(f"  {division}:")
        for machine, tasks in machines.items():
            if tasks:  # Only print if there are tasks
                print(f"    {machine}:")
                for task_time, item, machine_number in tasks:
                    print(f"      - {task_time.strftime('%H:%M')}: {item} (Machine {machine_number})")
            else:
                print(f"    {machine}: No tasks scheduled.")


# Example usage
today = datetime.now()
next_week = today + timedelta(days=7)

special_items_input = get_special_menu_input()

for date in [today, next_week]:
    schedule = schedule_tasks(divisions, special_items_input, total_machinery)  # Ensure correct parameters
    print("Generated Schedule:", schedule)  # Debugging line
    print_schedule(schedule, date)
    create_gantt_chart(schedule, date)

Generated Schedule: defaultdict(<function generate_initial_schedule.<locals>.<lambda> at 0x000001F6342FAA20>, {'Baking': defaultdict(<class 'list'>, {'Mixer': [(datetime.datetime(2025, 1, 16, 4, 0, 54, 857272), 'Bread', 1), (datetime.datetime(2025, 1, 16, 4, 0, 54, 857272), 'Croissant', 2), (datetime.datetime(2025, 1, 16, 4, 0, 54, 857272), 'Danish Pastry', 3), (datetime.datetime(2025, 1, 16, 4, 0, 54, 857272), 'Baguette', 4)], 'Proofer': [(datetime.datetime(2025, 1, 16, 4, 10, 54, 857272), 'Bread', 1), (datetime.datetime(2025, 1, 16, 4, 8, 54, 857272), 'Croissant', 2), (datetime.datetime(2025, 1, 16, 4, 48, 54, 857272), 'Danish Pastry', 2), (datetime.datetime(2025, 1, 16, 5, 20, 54, 857272), 'Baguette', 1)], 'Oven': [(datetime.datetime(2025, 1, 16, 5, 10, 54, 857272), 'Bread', 1), (datetime.datetime(2025, 1, 16, 4, 38, 54, 857272), 'Croissant', 2), (datetime.datetime(2025, 1, 16, 5, 33, 54, 857272), 'Danish Pastry', 3), (datetime.datetime(2025, 1, 16, 5, 35, 54, 857272), 'Baguette', 2

Generated Schedule: defaultdict(<function generate_initial_schedule.<locals>.<lambda> at 0x000001F60B0A2480>, {'Baking': defaultdict(<class 'list'>, {'Mixer': [(datetime.datetime(2025, 1, 16, 4, 0, 55, 46448), 'Bread', 1), (datetime.datetime(2025, 1, 16, 4, 0, 55, 46448), 'Croissant', 2), (datetime.datetime(2025, 1, 16, 4, 0, 55, 46448), 'Danish Pastry', 3), (datetime.datetime(2025, 1, 16, 4, 0, 55, 46448), 'Baguette', 4)], 'Proofer': [(datetime.datetime(2025, 1, 16, 4, 10, 55, 46448), 'Bread', 1), (datetime.datetime(2025, 1, 16, 4, 8, 55, 46448), 'Croissant', 2), (datetime.datetime(2025, 1, 16, 4, 48, 55, 46448), 'Danish Pastry', 2), (datetime.datetime(2025, 1, 16, 5, 20, 55, 46448), 'Baguette', 1)], 'Oven': [(datetime.datetime(2025, 1, 16, 5, 10, 55, 46448), 'Bread', 1), (datetime.datetime(2025, 1, 16, 4, 38, 55, 46448), 'Croissant', 2), (datetime.datetime(2025, 1, 16, 5, 33, 55, 46448), 'Danish Pastry', 3), (datetime.datetime(2025, 1, 16, 5, 35, 55, 46448), 'Baguette', 2)]}), 'Pastr

## time & space complexity, BIG O

In [3]:
from datetime import datetime, timedelta
import pandas as pd
import random
from collections import defaultdict
import heapq
import plotly.express as px
import math

import time
import tracemalloc

# FIRSTTTTTTT
# Define divisions and set menu items (exclusive to each division)
divisions = {
    "Baking": ["Bread", "Croissant", "Danish Pastry", "Baguette"],
    "Pastry": ["Cake", "Pie", "Cookies", "Brownies"],
    "Confectionery": ["Macarons", "Cupcakes", "Muffins", "Donuts", "Bagels"],
    "Butchery": ["Quiche"]
}

# Define total available machinery
total_machinery = {
    "Mixer": 4,
    "Proofer": 2,
    "Oven": 3
}

# Define unit times for each machine and item (in minutes)
unit_times = {
    "Mixer": {
        "Bread": 10,
        "Croissant": 8,
        "Danish Pastry": 12,
        "Cake": 15,
        "Pie": 20,
        "Cookies": 5,
        "Brownies": 8,
        "Macarons": 3,
        "Cupcakes": 4,
        "Muffins": 5,
        "Donuts": 6,
        "Bagels": 7,
        "Quiche": 10,
        "Baguette": 15
    },
    "Oven": {
        "Bread": 45,
        "Croissant": 25,
        "Danish Pastry": 30,
        "Cake": 60,
        "Pie": 40,
        "Cookies": 15,
        "Brownies": 20,
        "Macarons": 10,
        "Cupcakes": 12,
        "Muffins": 10,
        "Donuts": 8,
        "Bagels": 15,
        "Quiche": 30,
        "Baguette": 15
    },
    "Proofer": {
        "Bread": 60,
        "Croissant": 30,
        "Danish Pastry": 45,
        "Cake": 0,
        "Pie": 0,
        "Cookies": 0,
        "Brownies": 0,
        "Macarons": 0,
        "Cupcakes": 0,
        "Muffins": 0,
        "Donuts": 15,
        "Bagels": 45,
        "Quiche": 0,
        "Baguette": 15
    }
}


# Define machine downtime (in minutes)
downtime = {
    "Mixer": 15,
    "Oven": 30,
    "Proofer": 10,
}

def get_special_menu_input():
    special_items = []
    while True:
        item_name = input("Enter special menu item (type 'done' to finish): ")
        if item_name.lower() == "done":
            break

        print("Enter Division Responsible for this Menu item:")
        for i, division_name in enumerate(divisions.keys(), 1):  # Print divisions with numbers
            print(f"{i}. {division_name}")

        try:
            division_choice = int(input("Choose a division (1-4): "))
            if division_choice not in range(1, len(divisions) + 1):  # Check against valid range
                print("Invalid division. Please enter a valid division.")
                continue
            division = list(divisions.keys())[division_choice - 1]  # Get division name by index
        except (ValueError, IndexError):
            print("Invalid input. Please enter a number between 1 and 4.")
            continue

        processing_times = {}
        for machine in total_machinery.keys():
            try:
                processing_time = int(input(f"Enter processing time for {machine} in minutes: "))
                if processing_time < 0:
                    raise ValueError
                processing_times[machine] = processing_time
            except ValueError:
                print("Invalid input. Please enter a valid positive number.")
                continue

        special_items.append({
            "name": item_name,
            "division": division,
            "processing_times": processing_times
        })

    return special_items

def schedule_tasks(divisions, special_items, machines):
    # Initial solution: a random schedule
    # Pass machine_order to generate_initial_schedule
    machine_order = ["Mixer", "Proofer", "Oven"]  # Or any other desired order
    current_schedule = generate_initial_schedule(divisions, special_items, machines, machine_order)
    best_schedule = current_schedule
    best_cost = calculate_cost(best_schedule)

    if best_cost is None:
        raise ValueError("Initial best cost is None. Check the calculate_cost function.")

    initial_temp = 1000  # Initial temperature
    cooling_rate = 0.95   # Cooling rate
    temp = initial_temp

    while temp > 1:
        # Generate a new schedule by slightly modifying the current schedule
        new_schedule = modify_schedule(current_schedule)

        # Calculate the cost of the new schedule
        new_cost = calculate_cost(new_schedule)

        if new_cost is None:
            raise ValueError("New cost is None. Check the calculate_cost function.")

        # Decide whether to accept the new schedule
        if accept_solution(best_cost, new_cost, temp):
            current_schedule = new_schedule
            if new_cost < best_cost:
                best_schedule = new_schedule
                best_cost = new_cost

        # Cool down the temperature
        temp *= cooling_rate

    return best_schedule

def generate_initial_schedule(divisions, special_items, machines, machine_order):
    schedule = defaultdict(lambda: defaultdict(list))
    start_time = datetime.now().replace(hour=4, minute=0)  # Starting at 4 AM
    machine_order = ["Mixer", "Proofer", "Oven"]  # Define the required machine sequence

    # Machine availability tracking (key: machine type, value: list of available times for each machine)
    machine_times = {machine: [(start_time, i) for i in range(machines[machine])] for machine in machine_order}

    # Convert to heaps for efficient retrieval of earliest available time
    for machine in machine_times:
        heapq.heapify(machine_times[machine])

    # Combine regular items and special items
    for division, items in divisions.items():
        all_items = items + [item['name'] for item in special_items if item['division'] == division]
        for item in all_items:
            # Schedule the item on the required machines in order
            previous_task_completion_time = start_time  # Initialize for dependency tracking

            for machine in machine_order:
                duration = unit_times[machine].get(item, 0)  # Get processing time
                if duration > 0:  # Schedule only if processing time is greater than 0
                    # Find the earliest available machine of this type
                    machine_available_time, machine_index = heapq.heappop(machine_times[machine])
                    task_start_time = max(previous_task_completion_time, machine_available_time)

                    # Schedule the task
                    schedule[division][machine].append((task_start_time, item, machine_index + 1))

                    # Update machine availability, incorporating the processing time for THIS machine
                    task_completion_time = task_start_time + timedelta(minutes=duration)
                    heapq.heappush(machine_times[machine], (task_completion_time + timedelta(minutes=downtime[machine]), machine_index))

                    # Update previous_task_completion_time for the NEXT task of THIS ITEM
                    previous_task_completion_time = task_completion_time
    # --- Added Check to ensure all items are in the schedule ---
    all_menu_items = [item for sublist in divisions.values() for item in sublist] + [item['name'] for item in special_items]
    scheduled_items = [item for division_schedule in schedule.values() for machine_schedule in division_schedule.values() for _, item, _ in machine_schedule]

    missing_items = set(all_menu_items) - set(scheduled_items)
    if missing_items:
        print("WARNING: The following items were not scheduled:", missing_items)

    return schedule

def modify_schedule(schedule):
    # Convert the schedule to a list of tasks for modification
    tasks = []
    for division, machines in schedule.items():
        for machine, task_list in machines.items():
            tasks.extend(task_list)

    # Randomly swap two tasks
    if len(tasks) > 1:
        idx1, idx2 = random.sample(range(len(tasks)), 2)
        tasks[idx1], tasks[idx2] = tasks[idx2], tasks[idx1]  # Swap tasks

    # Rebuild the schedule from modified tasks
    new_schedule = defaultdict(lambda: defaultdict(list))
    for task in tasks:
        # Extract division and machine information from the original schedule
        division = None
        machine = None
        for div, machines in schedule.items():
            for mach, task_list in machines.items():
                if task in task_list:
                    division = div
                    machine = mach
                    break
            if division is not None:
                break

        if division is not None and machine is not None:
            task_time, item, machine_number = task
            new_schedule[division][machine].append((task_time, item, machine_number))
        else:
            print("WARNING: Task not found in original schedule:", task)

    return new_schedule

def accept_solution(best_cost, new_cost, temp):
    """
    Determines whether to accept a new solution based on the Metropolis criterion.

    Args:
        best_cost (float): The cost of the best solution found so far.
        new_cost (float): The cost of the new solution.
        temp (float): The current temperature.

    Returns:
        bool: True if the new solution is accepted, False otherwise.
    """
    if new_cost < best_cost:
        return True  # Always accept if the new solution is better
    else:
        # Accept with a probability that decreases with temperature
        acceptance_probability = math.exp((best_cost - new_cost) / temp)
        return random.random() < acceptance_probability

def calculate_cost(schedule):
    total_cost = 0  # Initialize total cost

    # Iterate through the schedule to calculate total processing time
    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Get the processing time for the item on the machine
                processing_time = unit_times[machine].get(item, 0)
                # Add processing time to total cost
                total_cost += processing_time

                # Add downtime after each task
                total_cost += downtime[machine]

    return total_cost



def create_gantt_chart(schedule, date):
    """
    Creates a Gantt chart from the schedule_tasks output, including downtime in red.

    Args:
        schedule (dict): The output from schedule_tasks.
        date (datetime): The date for which the schedule is generated.
    """
    df = []

    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Calculate the finish time based on unit_times
                finish_time = task_time + timedelta(minutes=unit_times.get(machine, {}).get(item, 0))

                # Append tasks to the DataFrame
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=task_time,
                    Finish=finish_time,
                    Item=item,
                    Type="Task"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

                # Add downtime after the task
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=finish_time,
                    Finish=finish_time + timedelta(minutes=downtime[machine]),
                    Item="Downtime",
                    Type="Downtime"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

    # Convert to DataFrame
    df = pd.DataFrame(df)  #Convert the list of dictionaries to a Pandas DataFrame

    # Check if the DataFrame is empty
    if df.empty:
        # Create a DataFrame with the needed columns and a single empty row if empty.
        df = pd.DataFrame(columns=["Machine", "Start", "Finish", "Item", "Type"])
        df.loc[0] = [None, None, None, None, None]

    #The rest of your function as it was
    # Ensure Start and Finish are datetime objects
    df['Start'] = pd.to_datetime(df['Start'])
    df['Finish'] = pd.to_datetime(df['Finish'])


    # Create the Gantt chart using Plotly Express
    # Define the colors for tasks and downtime
    color_map = {"Downtime": "grey"}  # Set downtime color to black

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Item",
                      color_discrete_map=color_map,
                      title=f"Machine Workload Visualization (Simulated Annealing) for {date.strftime('%Y-%m-%d')}",
                      labels={"Item": "Menu Item", "Machine": "Machine Index"})
    # Update layout for better visibility
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_traces(marker=dict(line=dict(width=0)))

    fig.show()

def create_dataframe(schedule):
    # Implementation for creating a DataFrame from the schedule
    pass


def print_schedule(schedule, date):
    """Prints the schedule for a given date."""
    print(f"\nSchedule for {date.strftime('%Y-%m-%d')}:")
    for division, machines in schedule.items():
        print(f"  {division}:")
        for machine, tasks in machines.items():
            if tasks:  # Only print if there are tasks
                print(f"    {machine}:")
                for task_time, item, machine_number in tasks:
                    print(f"      - {task_time.strftime('%H:%M')}: {item} (Machine {machine_number})")
            else:
                print(f"    {machine}: No tasks scheduled.")

def measure_total_performance_simulated_annealing(schedule_function, divisions, special_items, machines, dates):
    """
    Measures the total time and space complexity of the simulated annealing scheduling algorithm for multiple dates.

    Args:
        schedule_function (function): The scheduling function to measure.
        divisions (dict): The divisions and their items.
        special_items (list): The special items to schedule.
        machines (dict): The available machinery.
        dates (list): List of dates to run the scheduling function for.

    Returns:
        dict: Total time taken (in milliseconds) and peak memory usage (in bytes).
    """
    total_time_ms = 0
    total_peak_memory = 0

    for date in dates:
        # Start measuring memory
        tracemalloc.start()

        # Start measuring time in milliseconds
        start_time = time.time()

        # Execute the scheduling function
        schedule_function(divisions, special_items, machines)

        # End measuring time in milliseconds
        end_time = time.time()

        # Stop measuring memory
        current, peak_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        # Accumulate total time and memory
        total_time_ms += ((end_time - start_time) * 1000)  # Convert to milliseconds
        total_peak_memory = max(total_peak_memory, peak_memory)  # Use peak memory across all runs

    return {
        "Total Time Taken (ms)": total_time_ms,
        "Peak Memory Usage (bytes)": total_peak_memory
    }

# Get special menu items from user input
special_items_input = get_special_menu_input()

# Define today's date and next week's date
today = datetime.now().date()  # Get today's date
next_week = today + timedelta(weeks=1)  # Get the date for next week

# Example usage
dates = [today, next_week]  # List of dates for performance measurement

# Measure performance for the simulated annealing approach
performance_simulated_annealing = measure_total_performance_simulated_annealing(schedule_tasks, divisions, special_items_input, total_machinery, dates)

# Print total performance metrics for simulated annealing
print(f"SIMULATED ANNEALING")
print(f"Total Performance Metrics for {len(dates)} days:")
print(f"  Total Time Taken: {performance_simulated_annealing['Total Time Taken (ms)']} ms")
print(f"  Peak Memory Usage: {performance_simulated_annealing['Peak Memory Usage (bytes)']} bytes")

SIMULATED ANNEALING
Total Performance Metrics for 2 days:
  Total Time Taken: 248.12841415405273 ms
  Peak Memory Usage: 11720 bytes


# unoptimized (  Basic Code )

In [2]:
from datetime import datetime, timedelta
import pandas as pd
import random
from collections import defaultdict
import heapq
import plotly.express as px
import math

# Define total available machinery
total_machinery = {
    "Mixer": 4,
    "Proofer": 2,
    "Oven": 3
}

# Define divisions and their menu items
divisions = {
    1: ("Baking", ["Bread", "Croissant", "Danish Pastry"]),
    2: ("Pastry", ["Cake", "Pie", "Cookies", "Brownies"]),
    3: ("Confectionery", ["Macarons", "Cupcakes", "Muffins", "Donuts", "Bagels"]),
    4: ("Butchery", ["Quiche"])
}

# Define unit times for each machine and item (in minutes)
unit_times = {
    "Mixer": {
        "Bread": 10,
        "Croissant": 8,
        "Danish Pastry": 12,
        "Cake": 15,
        "Pie": 20,
        "Cookies": 5,
        "Brownies": 8,
        "Macarons": 3,
        "Cupcakes": 4,
        "Muffins": 5,
        "Donuts": 6,
        "Bagels": 7,
        "Quiche": 10
    },
    "Oven": {
        "Bread": 45,
        "Croissant": 25,
        "Danish Pastry": 30,
        "Cake": 60,
        "Pie": 40,
        "Cookies": 15,
        "Brownies": 20,
        "Macarons": 10,
        "Cupcakes": 12,
        "Muffins": 10,
        "Donuts": 8,
        "Bagels": 15,
        "Quiche": 30
    },
    "Proofer": {
        "Bread": 60,
        "Croissant": 30,
        "Danish Pastry": 45,
        "Cake": 0,
        "Pie": 0,
        "Cookies": 0,
        "Brownies": 0,
        "Macarons": 0,
        "Cupcakes": 0,
        "Muffins": 0,
        "Donuts": 15,
        "Bagels": 45,
        "Quiche": 0
    }
}

# Define machine downtime (in minutes)
downtime = {
    "Mixer": 15,
    "Oven": 30,
    "Proofer": 10,
}

def get_special_menu_input():
    special_items = []
    while True:
        item_name = input("Enter special menu item (type 'done' to finish): ")
        if item_name.lower() == "done":
            break

        print("Enter Division Responsible for this Menu item:")
        print("1. Baking")
        print("2. Pastry")
        print("3. Confectionery")
        print("4. Butchery")

        try:
            division_choice = int(input("Choose a division (1-4): "))
            if division_choice not in divisions:
                print("Invalid division. Please enter a valid division.")
                continue
            division = divisions[division_choice][0]
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 4.")
            continue

        processing_times = {}
        for machine in total_machinery.keys():
            try:
                processing_time = int(input(f"Enter processing time for {machine} in minutes: "))
                if processing_time < 0:
                    raise ValueError
                processing_times[machine] = processing_time
            except ValueError:
                print("Invalid input. Please enter a valid positive number.")
                continue

        special_items.append({
            "name": item_name,
            "division": division,
            "processing_times": processing_times
        })

    return special_items

def schedule_tasks(date, special_items_input):
    """Schedules tasks for a given date, considering shared machinery and dependencies."""
    schedule = defaultdict(lambda: defaultdict(list))
    start_time = datetime.combine(date, datetime.min.time()) + timedelta(hours=4)

    # Initialize machine availability times
    machine_times = {machine: [start_time] * total_machinery[machine] for machine in total_machinery}
    machine_order = ["Mixer", "Proofer", "Oven"]

    # Schedule set menu items and special items
    for division, division_items in divisions.values():
        all_items = division_items + [f"Special: {item['name']}" for item in special_items_input if item['division'] == division]
        random.shuffle(all_items)

        for item in all_items:
            item_name = item if not item.startswith("Special: ") else item[9:]
            processing_time = {machine: unit_times[machine].get(item_name, 0) for machine in machine_order}

            if "Special: " in item:
                special_item = next((i for i in special_items_input if i["name"] == item_name), None)
                if special_item:
                    processing_time.update(special_item["processing_times"])

            last_finish_time = start_time  # Start time for the first machine
            for machine in machine_order:
                processing_duration = processing_time[machine]
                if processing_duration > 0:
                    # Find the next available time for the current machine
                    machine_available_time = min(machine_times[machine])
                    machine_index = machine_times[machine].index(machine_available_time)

                    # The task can start as soon as it's finished on the previous machine
                    task_start_time = max(last_finish_time, machine_available_time)

                    # Schedule the task
                    schedule[division][machine].append((task_start_time, item, machine_index + 1))

                    # Calculate finish time and update machine availability (accounting for downtime)
                    finish_time = task_start_time + timedelta(minutes=processing_duration)
                    machine_times[machine][machine_index] = finish_time + timedelta(minutes=downtime[machine])

                    # Update last_finish_time for the next machine (without adding downtime)
                    last_finish_time = finish_time

    return schedule

def create_gantt_chart(schedule, date):
    """
    Creates a Gantt chart from the schedule_tasks output, including downtime in red.

    Args:
        schedule (dict): The output from schedule_tasks.
        date (datetime): The date for which the schedule is generated.
    """
    df = []

    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Calculate the finish time based on unit_times
                finish_time = task_time + timedelta(minutes=unit_times.get(machine, {}).get(item, 0))

                # Append tasks to the DataFrame
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=task_time,
                    Finish=finish_time,
                    Item=item,
                    Type="Task"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

                # Add downtime after the task
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=finish_time,
                    Finish=finish_time + timedelta(minutes=downtime[machine]),
                    Item="Downtime",
                    Type="Downtime"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

    # Convert to DataFrame
    df = pd.DataFrame(df)  #Convert the list of dictionaries to a Pandas DataFrame

    # Check if the DataFrame is empty
    if df.empty:
        # Create a DataFrame with the needed columns and a single empty row if empty.
        df = pd.DataFrame(columns=["Machine", "Start", "Finish", "Item", "Type"])
        df.loc[0] = [None, None, None, None, None]

    #The rest of your function as it was
    # Ensure Start and Finish are datetime objects
    df['Start'] = pd.to_datetime(df['Start'])
    df['Finish'] = pd.to_datetime(df['Finish'])


    # Create the Gantt chart using Plotly Express
    # Define the colors for tasks and downtime
    color_map = {"Downtime": "grey"}  # Set downtime color to black

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Item",
                      color_discrete_map=color_map,
                      title=f"Machine Workload Visualization (Basic) for {date.strftime('%Y-%m-%d')}",
                      labels={"Item": "Menu Item", "Machine": "Machine Index"})
    # Update layout for better visibility
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_traces(marker=dict(line=dict(width=0)))

    fig.show()


def print_schedule(schedule, date):
    """Prints the schedule for a given date."""
    print(f"\nSchedule for {date.strftime('%Y-%m-%d')}:")
    for division, machines in schedule.items():
        print(f"  {division}:")
        for machine, tasks in machines.items():
            print(f"    {machine}:")
            for task_time, item, machine_number in tasks:
                print(f"      - {task_time.strftime('%H:%M')}: {item} (Machine {machine_number})")

# Example usage
today = datetime.now()
next_week = today + timedelta(days=7)

special_items_input = []

for date in [today, next_week]:
    schedule = schedule_tasks(date, special_items_input)
    # print("Generated Schedule:", schedule)  # Debugging line
    # print_schedule(schedule, date)
    create_gantt_chart(schedule, date)

## BIG O, time & space complexity ( basic code )

In [33]:
from datetime import datetime, timedelta
import pandas as pd
import random
from collections import defaultdict
import heapq
import plotly.express as px
import math

# Define total available machinery
total_machinery = {
    "Mixer": 4,
    "Proofer": 2,
    "Oven": 3
}

# Define divisions and their menu items
divisions = {
    1: ("Baking", ["Bread", "Croissant", "Danish Pastry"]),
    2: ("Pastry", ["Cake", "Pie", "Cookies", "Brownies"]),
    3: ("Confectionery", ["Macarons", "Cupcakes", "Muffins", "Donuts", "Bagels"]),
    4: ("Butchery", ["Quiche"])
}

# Define unit times for each machine and item (in minutes)
unit_times = {
    "Mixer": {
        "Bread": 10,
        "Croissant": 8,
        "Danish Pastry": 12,
        "Cake": 15,
        "Pie": 20,
        "Cookies": 5,
        "Brownies": 8,
        "Macarons": 3,
        "Cupcakes": 4,
        "Muffins": 5,
        "Donuts": 6,
        "Bagels": 7,
        "Quiche": 10
    },
    "Oven": {
        "Bread": 45,
        "Croissant": 25,
        "Danish Pastry": 30,
        "Cake": 60,
        "Pie": 40,
        "Cookies": 15,
        "Brownies": 20,
        "Macarons": 10,
        "Cupcakes": 12,
        "Muffins": 10,
        "Donuts": 8,
        "Bagels": 15,
        "Quiche": 30
    },
    "Proofer": {
        "Bread": 60,
        "Croissant": 30,
        "Danish Pastry": 45,
        "Cake": 0,
        "Pie": 0,
        "Cookies": 0,
        "Brownies": 0,
        "Macarons": 0,
        "Cupcakes": 0,
        "Muffins": 0,
        "Donuts": 15,
        "Bagels": 45,
        "Quiche": 0
    }
}

# Define machine downtime (in minutes)
downtime = {
    "Mixer": 15,
    "Oven": 30,
    "Proofer": 10,
}

def get_special_menu_input():
    special_items = []
    while True:
        item_name = input("Enter special menu item (type 'done' to finish): ")
        if item_name.lower() == "done":
            break

        print("Enter Division Responsible for this Menu item:")
        print("1. Baking")
        print("2. Pastry")
        print("3. Confectionery")
        print("4. Butchery")

        try:
            division_choice = int(input("Choose a division (1-4): "))
            if division_choice not in divisions:
                print("Invalid division. Please enter a valid division.")
                continue
            division = divisions[division_choice][0]
        except ValueError:
            print("Invalid input. Please enter a number between 1 and 4.")
            continue

        processing_times = {}
        for machine in total_machinery.keys():
            try:
                processing_time = int(input(f"Enter processing time for {machine} in minutes: "))
                if processing_time < 0:
                    raise ValueError
                processing_times[machine] = processing_time
            except ValueError:
                print("Invalid input. Please enter a valid positive number.")
                continue

        special_items.append({
            "name": item_name,
            "division": division,
            "processing_times": processing_times
        })

    return special_items

def schedule_tasks(date, special_items_input):
    """Schedules tasks for a given date, considering shared machinery and dependencies."""
    schedule = defaultdict(lambda: defaultdict(list))
    start_time = datetime.combine(date, datetime.min.time()) + timedelta(hours=4)

    # Initialize machine availability times
    machine_times = {machine: [start_time] * total_machinery[machine] for machine in total_machinery}
    machine_order = ["Mixer", "Proofer", "Oven"]

    # Schedule set menu items and special items
    for division, division_items in divisions.values():
        all_items = division_items + [f"Special: {item['name']}" for item in special_items_input if item['division'] == division]
        random.shuffle(all_items)

        for item in all_items:
            item_name = item if not item.startswith("Special: ") else item[9:]
            processing_time = {machine: unit_times[machine].get(item_name, 0) for machine in machine_order}

            if "Special: " in item:
                special_item = next((i for i in special_items_input if i["name"] == item_name), None)
                if special_item:
                    processing_time.update(special_item["processing_times"])

            last_finish_time = start_time  # Start time for the first machine
            for machine in machine_order:
                processing_duration = processing_time[machine]
                if processing_duration > 0:
                    # Find the next available time for the current machine
                    machine_available_time = min(machine_times[machine])
                    machine_index = machine_times[machine].index(machine_available_time)

                    # The task can start as soon as it's finished on the previous machine
                    task_start_time = max(last_finish_time, machine_available_time)

                    # Schedule the task
                    schedule[division][machine].append((task_start_time, item, machine_index + 1))

                    # Calculate finish time and update machine availability (accounting for downtime)
                    finish_time = task_start_time + timedelta(minutes=processing_duration)
                    machine_times[machine][machine_index] = finish_time + timedelta(minutes=downtime[machine])

                    # Update last_finish_time for the next machine (without adding downtime)
                    last_finish_time = finish_time

    return schedule

def create_gantt_chart(schedule, date):
    """
    Creates a Gantt chart from the schedule_tasks output, including downtime in red.

    Args:
        schedule (dict): The output from schedule_tasks.
        date (datetime): The date for which the schedule is generated.
    """
    df = []

    for division, machines in schedule.items():
        for machine, tasks in machines.items():
            for task_time, item, machine_number in tasks:
                # Calculate the finish time based on unit_times
                finish_time = task_time + timedelta(minutes=unit_times.get(machine, {}).get(item, 0))

                # Append tasks to the DataFrame
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=task_time,
                    Finish=finish_time,
                    Item=item,
                    Type="Task"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

                # Add downtime after the task
                df.append(dict(
                    Machine=f"{machine} {machine_number}",
                    Start=finish_time,
                    Finish=finish_time + timedelta(minutes=downtime[machine]),
                    Item="Downtime",
                    Type="Downtime"  # Add a 'Type' column to distinguish between tasks and downtime
                ))

    # Convert to DataFrame
    df = pd.DataFrame(df)  #Convert the list of dictionaries to a Pandas DataFrame

    # Check if the DataFrame is empty
    if df.empty:
        # Create a DataFrame with the needed columns and a single empty row if empty.
        df = pd.DataFrame(columns=["Machine", "Start", "Finish", "Item", "Type"])
        df.loc[0] = [None, None, None, None, None]

    #The rest of your function as it was
    # Ensure Start and Finish are datetime objects
    df['Start'] = pd.to_datetime(df['Start'])
    df['Finish'] = pd.to_datetime(df['Finish'])


    # Create the Gantt chart using Plotly Express
    # Define the colors for tasks and downtime
    color_map = {"Downtime": "grey"}  # Set downtime color to black

    fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Item",
                      color_discrete_map=color_map,
                      title=f"Machine Workload Visualization (Basic) for {date.strftime('%Y-%m-%d')}",
                      labels={"Item": "Menu Item", "Machine": "Machine Index"})
    # Update layout for better visibility
    fig.update_yaxes(categoryorder="total ascending")
    fig.update_traces(marker=dict(line=dict(width=0)))

    fig.show()


def print_schedule(schedule, date):
    """Prints the schedule for a given date."""
    print(f"\nSchedule for {date.strftime('%Y-%m-%d')}:")
    for division, machines in schedule.items():
        print(f"  {division}:")
        for machine, tasks in machines.items():
            print(f"    {machine}:")
            for task_time, item, machine_number in tasks:
                print(f"      - {task_time.strftime('%H:%M')}: {item} (Machine {machine_number})")

def measure_total_performance(schedule_function, dates, *args):
    """
    Measures the total time and space complexity of the scheduling algorithm for multiple dates.

    Args:
        schedule_function (function): The scheduling function to measure.
        dates (list): List of dates to run the scheduling function for.
        *args: Additional arguments to pass to the scheduling function.

    Returns:
        dict: Total time taken (in nanoseconds) and peak memory usage (in bytes).
    """
    total_time_ms = 0
    total_peak_memory = 0

    for date in dates:
        # Start measuring memory
        tracemalloc.start()

        # Start measuring time in nanoseconds
        start_time_ms = time.time()

        # Execute the scheduling function
        schedule_function(date, *args)

        # End measuring time in nanoseconds
        end_time_ms = time.time()

        # Stop measuring memory
        current, peak_memory = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        # Accumulate total time and memory
        total_time_ms += ((end_time_ms - start_time_ms) * 1000)
        total_peak_memory = max(total_peak_memory, peak_memory)  # Use peak memory across all runs

    return {
        "Total Time Taken (ms)": total_time_ms,
        "Peak Memory Usage (bytes)": total_peak_memory
    }

# Example usage
today = datetime.now()
next_week = today + timedelta(days=7)

special_items_input = []

# Measure performance
dates = [today, next_week]
performance = measure_total_performance(schedule_tasks, dates, special_items_input)

# Print total performance metrics
print(f"Total Performance Metrics for {len(dates)} days:")
print(f"  Total Time Taken: {performance['Total Time Taken (ms)']} ms")
print(f"  Peak Memory Usage: {performance['Peak Memory Usage (bytes)']} bytes")

for date in dates:
    schedule = schedule_tasks(date, special_items_input)
    create_gantt_chart(schedule, date)

Total Performance Metrics for 2 days:
  Total Time Taken: 2.2554397583007812 ms
  Peak Memory Usage: 2606005 bytes
